In [10]:
%pip install langchain langchain_community langchain-text-splitters faiss-cpu sentence-transformers huggingface-hub py pdf streamlit langgraph pandas langchain-groq

  Using cached py-1.11.0-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached pdf-2020.11.12-py3-none-any.whl.metadata (789 bytes)
  Using cached pdftotree-0.5.0-py3-none-any.whl.metadata (10 kB)
  Using cached pdfminer-20191125.tar.gz (4.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached silverware-2020.11.12-py3-none-any.whl.metadata (771 bytes)
  Using cached disk-2021.6.29-py3-none-any.whl.metadata (3.6 kB)
  Using cached chronometry-2020.11.12-py3-none-any.whl.metadata (2.4 kB)
  Using cached slytherin-2020.11.12-py3-none-any.whl.metadata (2.3 kB)
  Using cached colouration-2020.7.26-py3-none-any.whl.metadata (3.6 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [11]:


import os
import pandas as pd
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langgraph.graph import StateGraph , START , END
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from typing import Dict, Any, TypedDict
   

In [12]:
claims = pd.read_csv('/Users/prabhat/OneDrive/Desktop/codingbase/PYTHON_Codebase/AI_WARAANTY_DETECTOR/Data/warranty_claims - Copy.csv')


loader = PyPDFLoader('/Users/prabhat/OneDrive/Desktop/codingbase/PYTHON_Codebase/AI_WARAANTY_DETECTOR/Data/AutoDrive_Warranty_Policy_2025.pdf')
load= loader.load()
policy_text = " ".join([doc.page_content for doc in load])

In [18]:
%pip install langchain-groq

  Using cached langchain_groq-1.0.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached groq-0.33.0-py3-none-any.whl.metadata (16 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.chat_models import init_chat_model

llm =  init_chat_model ("openai/gpt-oss-20b",model_provider='GROQ',api_key = '')

In [21]:
class ClassState(TypedDict):
    claims:Dict[str,Any]
    policy_tracker:str
    fraud_tracker:str
    fraud_score:float
    evidence:str
    decision:str

In [22]:
def policy_check_agent(State: ClassState) -> ClassState:
    claim = State["claims"]

    # ✅ Safely get Vehicle Type (use default if missing)
    vehicle_type = claim.get("Vehicle Type", "Unknown")

    # ✅ Infer vtype based on value
    if vehicle_type.lower() == "car":
        vtype = "Four Wheeler"
    elif vehicle_type.lower() in ["bike", "scooter", "motorcycle"]:
        vtype = "Two Wheeler"
    else:
        vtype = "Unknown"

    # Build your prompt
    prompt = f"""
    You are an expert insurance policy analyst. Analyze the following warranty claim details against the provided warranty policy document.

    Policy manual:
    {policy_text}

    Warranty claim details:
    vtype: {vtype}
    claim: {claim}

    Based on the above warranty policy document and claim details, determine:
    Answer: Policy is Covered or Not Covered. Provide a brief explanation for your decision.
    """

    # Invoke LLM
    response = llm.invoke(prompt)

    # Store the result
    State["policy_tracker"] = response.content.strip()
    return State


In [23]:
def fraud_detection_agent(State: ClassState) -> ClassState:
    claim = State['claims']

    prompt = f"""
    You are an expert fraud detection analyst. Analyze the following warranty claim details for potential fraud indicators.
     warranty policy document :
        {policy_text}

    claim details:
    claim:{claim}

        Based on the above warranty policy document and claim details, assess the likelihood of fraud. Provide a fraud score between 0 (no fraud) to 1 (definite fraud) along with a brief explanation.

    """
    response = llm.invoke(prompt)
    try:
        score = float(response.content.strip())
    except :
        score=0.5
    State['evidence'] = response.content.strip()
    return State

In [24]:
def evidence_collector_agent(State: ClassState) -> ClassState:
    claim = State['claims']

    prompt = f"""
    You are an expert evidence collection analyst. Based on the following warranty claim details and previous analyses, summarize the key evidence supporting or refuting the claim.

     warranty policy document :
        {policy_text}

    claim details:
    claim:{claim}

    Previous Analyses:
    Policy Analysis: {State['policy_tracker']}
    Fraud Detector : {State['fraud_tracker']}
    Fraud Score: {State['fraud_score']}

     Based on the above information, summarize the key evidence supporting or refuting the claim.
        Answer:List any red flags, inconsistencies, or supporting details found in the claim.

    Provide a concise summary of the key evidence.

    """
    response = llm.invoke(prompt)
    State['evidence'] = response.content.strip()
    return State

In [25]:
def action_agent(State: ClassState) -> ClassState:
    claim = State['claims']
    policy_analysis = State['policy_tracker']
  
    evidence =  State['evidence']

    prompt = f"""
    You are an expert claims decision analyst. Based on the following warranty claim details and previous analyses, make a final decision on the claim.

     warranty policy document :
        {policy_text}

    claim details:
    claim:{claim}

    Previous Analyses:
    Policy Analysis: {State['policy_tracker']}
    
    Fraud Score: {State['fraud_score']}
    Evidence Summary: {State['evidence']}

     Based on the above information, make a final decision on the claim.
        Answer:Approve or Deny the claim. Provide a brief justification for your decision.

    """
    decision_text = ""
    try :
        if "approve" in decision_text.lower():
            decision_text = "Approve"   
        elif "deny" in decision_text.lower():
            decision_text = "DisApprove"

        elif "insufficient" in evidence.lower():
            decision_text = "Escalate for further review"

    except:
        decision_text = "Escalate for further review"


    response = llm.invoke(prompt)
    State['decision'] = response.content.strip()
    return State

In [26]:
graph = StateGraph(ClassState)

graph.add_node("PolicyCheck", policy_check_agent)
graph.add_node("FraudScoring", fraud_detection_agent)
graph.add_node("EvidenceCollector", evidence_collector_agent)
graph.add_node("Action", action_agent)

graph.set_entry_point("PolicyCheck")
graph.add_edge("PolicyCheck", "FraudScoring")
graph.add_edge("FraudScoring", "EvidenceCollector")
graph.add_edge("EvidenceCollector", "Action")
graph.add_edge("Action", END)

app = graph.compile()

In [27]:
results = []

for idx, row in claims.iterrows():
    state = app.invoke({"claims": row.to_dict(), 
                       "policy_tracker": "",
                       "fraud_tracker": "",
                       "fraud_score": 0.0,
                       "evidence": "",
                       "decision": ""})
    results.append({
        "claim_id": row["claim_id"],
        "model": row["model"],
        "decision": state["decision"],
        "policy_tracker": state["policy_tracker"],
        "fraud_score": state.get("fraud_score", 0.0),
        "evidence": state["evidence"]
    })

results_df = pd.DataFrame(results)


In [28]:
print(claims.columns.tolist())


['claim_id', 'model', 'purchase_date', 'claim_date', 'days_since_purchase', 'mileage', 'part_replaced', 'part_cost', 'labor_cost', 'total_cost', 'invoice_present', 'image_present', 'previous_claims']


In [29]:
results_df

,claim_id,model,decision,policy_tracker,fraud_score,evidence
0,CLM-DEMO-1,"Volt-X (Four-Wheeler, EV Sedan)",**Decision:** Deny\n\n**Justification:** \nTh...,**Answer: Not Covered**\n\n**Explanation:** \...,0.0,**Key Evidence Summary**\n\n| Evidence | Suppo...
1,CLM-DEMO-2,"Pulse-3 (Two-Wheeler, Sports Bike)",**Decision:** Deny the claim.\n\n**Justificati...,**Answer: Not Covered**\n\n**Reasoning**\n\n1....,0.0,**Key Evidence Summary – Claim CLM-DEMO-2**\n\...
2,CLM-DEMO-3,"Torque-7 (Four-Wheeler, SUV Diesel)",**Answer: Approve** \n\nThe vehicle is within...,**Answer:** Covered \n\n**Explanation:** \n-...,0.0,**Key Evidence Summary**\n\n| Evidence | Suppo...
3,CLM-DEMO-APPROVED,"Volt-X (Four-Wheeler, EV Sedan)",**Decision:** Approve\n\n**Justification:** \...,**Answer: Policy is Covered**\n\n**Explanation...,0.0,**Key Evidence Summary – Claim CLM‑DEMO‑APPROV...
